# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [19]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls 

In [20]:
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [21]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

# reshape to 2d feature
train = train.reshape(-1, 1, 11, 39)
test = test.reshape(-1, 1, 11, 39)

Loading data ...


In [22]:
print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Size of training data: (1229932, 1, 11, 39)
Size of testing data: (451552, 1, 11, 39)


## Create Dataset

In [23]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()

        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
        

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)



Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [24]:
VAL_RATIO=0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 1, 11, 39)
Size of validation set: (245987, 1, 11, 39)


## Normalization

In [25]:
tr = train_x[:, 0, 5].reshape(-1, 1, 1, 39)
mean = tr.mean(axis=0, keepdims=True)
std = tr.std(axis=0, keepdims=True)
norm = lambda x: (x-mean)/std
train_x = norm(train_x)
val_x = norm(val_x)
test = norm(test)

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [26]:
BATCH_SIZE = 256

from torch.utils.data import DataLoader


train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [27]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

174

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [28]:
import torch
import torch.nn as nn

In [29]:
class BlockConv2d(nn.Module):
    def __init__(self, ch_in, ch_out, k, act, use_bn=True, drop=0):
        super(BlockConv2d, self).__init__()
        list = [
            nn.Conv2d(ch_in, ch_out, k),
            act,
        ]
        if use_bn: list.append(nn.BatchNorm2d(ch_out))
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)

class BlockLinear(nn.Module):
    def __init__(self, ch_in, ch_out, act, use_bn=True, drop=0):
        super(BlockLinear, self).__init__()
        list = [
            nn.Linear(ch_in, ch_out),
            act,
        ]
        if use_bn: list.append(nn.BatchNorm1d(ch_out))
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)


In [34]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.act = nn.SELU()
        # self.act = nn.ReLU()

        # size = (1, 11, 39)
        self.n_0 = [1, 1024, 256] + [128]*3
        self.k_0 = [(1, 39)] + [(1, 1)] + [(3, 1)]*3
        self.l_0 = len(self.k_0)
        self.conv0 = [
            BlockConv2d(self.n_0[i], self.n_0[i+1], self.k_0[i], self.act, use_bn=True, drop=0.1)
            for i in range(self.l_0)]
        self.conv0 = nn.Sequential(*self.conv0)

        # size = (128, 9, 3)
        self.flat = nn.Flatten()

        # size = (9*9*3)
        self.n_1 = [128*5, 512, 256, 256, 256]
        self.l_1 = len(self.n_1)
        self.linears = [
            BlockLinear(self.n_1[i], self.n_1[i+1], self.act, use_bn=True, drop=0.1)
            for i in range(self.l_1-1)
        ]
        self.linears = nn.Sequential(*self.linears)

        self.out = nn.Linear(self.n_1[-1], 39)

    def forward(self, x):
        x = self.conv0(x)
        x = self.flat(x)
        x = self.linears(x)
        x = self.out(x)
        return x

## Training

In [31]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [32]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [35]:
# fix random seed for reproducibility
same_seeds(52728)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 50               # number of training epoch
learning_rate = 1e-4       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'
overfit_model_path = './overfit_model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
# model.load_state_dict(torch.load(overfit_model_path))
criterion = nn.CrossEntropyLoss() 
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

DEVICE: cuda


In [36]:
print(model)

Classifier(
  (act): SELU()
  (conv0): Sequential(
    (0): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(1, 1024, kernel_size=(1, 39), stride=(1, 1))
        (1): SELU()
        (2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): SELU()
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (2): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(256, 128, kernel_size=(3, 1), stride=(1, 1))
        (1): SELU()
        (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (3): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 1), s

In [37]:
gain = 3/4 # for selu
for n, p in model.named_parameters():
    print(n, p.shape)
    if p.dim() >= 2:
        print("normal init")
        nn.init.xavier_normal_(p, gain)

conv0.0.net.0.weight torch.Size([1024, 1, 1, 39])
normal init
conv0.0.net.0.bias torch.Size([1024])
conv0.0.net.2.weight torch.Size([1024])
conv0.0.net.2.bias torch.Size([1024])
conv0.1.net.0.weight torch.Size([256, 1024, 1, 1])
normal init
conv0.1.net.0.bias torch.Size([256])
conv0.1.net.2.weight torch.Size([256])
conv0.1.net.2.bias torch.Size([256])
conv0.2.net.0.weight torch.Size([128, 256, 3, 1])
normal init
conv0.2.net.0.bias torch.Size([128])
conv0.2.net.2.weight torch.Size([128])
conv0.2.net.2.bias torch.Size([128])
conv0.3.net.0.weight torch.Size([128, 128, 3, 1])
normal init
conv0.3.net.0.bias torch.Size([128])
conv0.3.net.2.weight torch.Size([128])
conv0.3.net.2.bias torch.Size([128])
conv0.4.net.0.weight torch.Size([128, 128, 3, 1])
normal init
conv0.4.net.0.bias torch.Size([128])
conv0.4.net.2.weight torch.Size([128])
conv0.4.net.2.bias torch.Size([128])
linears.0.net.0.weight torch.Size([512, 640])
normal init
linears.0.net.0.bias torch.Size([512])
linears.0.net.2.weight t

In [38]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')
else:
    torch.save(model.state_dict(), overfit_model_path)
    print('saving overfit model')

  0%|          | 1/3844 [00:00<07:53,  8.11it/s][001/050] Train Acc: 0.565967 Loss: 1.426780 | Val Acc: 0.661515 loss: 1.068313
saving model with acc 0.662
  0%|          | 1/3844 [00:00<07:19,  8.74it/s][002/050] Train Acc: 0.636561 Loss: 1.153132 | Val Acc: 0.690016 loss: 0.965876
saving model with acc 0.690
  0%|          | 1/3844 [00:00<07:24,  8.65it/s][003/050] Train Acc: 0.660130 Loss: 1.064399 | Val Acc: 0.702899 loss: 0.915769
saving model with acc 0.703
  0%|          | 1/3844 [00:00<07:25,  8.64it/s][004/050] Train Acc: 0.675873 Loss: 1.007235 | Val Acc: 0.713395 loss: 0.874585
saving model with acc 0.713
  0%|          | 1/3844 [00:00<07:25,  8.62it/s][005/050] Train Acc: 0.687808 Loss: 0.964939 | Val Acc: 0.720197 loss: 0.853370
saving model with acc 0.720
  0%|          | 1/3844 [00:00<07:20,  8.72it/s][006/050] Train Acc: 0.697678 Loss: 0.930312 | Val Acc: 0.726103 loss: 0.831733
saving model with acc 0.726
  0%|          | 1/3844 [00:00<07:32,  8.50it/s][007/050] Train 

KeyboardInterrupt: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [39]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [40]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [41]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))